In [853]:
import plotly.graph_objects as go
import math
import numpy as np


In [ ]:
class Order():
    def __init__(self,price,volume,isBuy,owner = "None"):
        self.price = price
        self.volume = volume
        self.isBuy = isBuy
        self.unfilledVolume = volume
        self.executionPrices = []
        self.owner = owner
        self.location = None

    def __str__(self):
        return f"Owner={self.owner}, Price={self.price}, Volume={self.volume}, Unfilled={self.unfilledVolume}"

    def execute(self,execution): #in the future can change this to log when orders are filled or execute something or whatever (amount,price)
        self.executionPrices.append(execution)
        self.unfilledVolume -= execution[0] 

        if self.isBuy:    
            self.owner.cash -= execution[0] * execution[1]
            self.owner.position += execution[0]
        else:
            self.owner.cash += execution[0] * execution[1]
            self.owner.position -= execution[0]

        if self.unfilledVolume == 0:
            self.owner.orders.remove(self)


    def remove_order(self):
        node = self.location
        if node != None: #if the location is none the order is unplaced.
            if self in node.orders:
                node.orders.remove(self)
                if len(node.orders) == 0: #deleting the node

                    if node.left != None:
                        cNode = node.left
                        while cNode != None:
                            cNode = cNode.right

                        node.left.parent = node.parent
                        node.parent.left = node.left

                        node.right.parent = cNode
                        cNode.right = node.right

                    else:
                        node.parent.left = node.right

            else:
                return False



            
            



In [ ]:
class Node():
    def __init__(self,value,orders = None):
        self.value = value
        self.left = None
        self.right = None
        self.parent = None
        if orders == None: #for increased efficiency change orders to a linked list but I am to lazy to do that right now
            self.orders = []
        else:
            self.orders = orders

    def __str__(self): #does inorder traversal returning all of the orders
        output  = []
        for order in self.orders:
            output.append(str(order))
        return ".\n".join(output)

    def in_order(self):
        if self.left != None:
            yield from self.left.in_order()

        yield self.orders,self
        
        if self.right != None:
            yield from self.right.in_order()


    def insert(self,order,direction): 
        if order.price == self.value:
            self.orders.append(order) 
            order.location = self

        elif order.price * direction < self.value * direction: #move to right node
            if self.right == None:
                self.right = Node(order.price,[order])
                self.right.parent = self
                order.location = self
            else:
                self.right.insert(order,direction)

        else: #move to left node
            if self.left == None:
                self.left = Node(order.price,[order])
                self.left.parent = self
                order.location = self
            else:
                self.left.insert(order,direction)



In [ ]:
class OrderTree():
    def __init__(self,isBuy,node = None):
        self.head = node
        self.isBuy = isBuy

    def bestPrice(self):
        if self.head == None:
            if self.isBuy:
                return -math.inf
            else:
                return math.inf
        else:
            currentNode = self.head
            while currentNode.left is not None:
                currentNode = currentNode.left
            return currentNode.value

    def __str__(self):
        header = f"bestPrice={self.bestPrice()}"
        if self.head == None:
            tree = "empty"
        else:
            tree = ".\n".join([str(order) for orders,_ in self.head.in_order() for order in orders])
        return header + '\n' + tree


    def insertOrder(self,order): 
        if self.head is None: #to optimize use a self balancing tree
            self.head = Node(order.price, [order])
        else:
            direction = 1 if self.isBuy else -1
            self.head.insert(order, direction)


In [857]:
class orderbook:
    def __init__(self):
        self.sells = OrderTree(False)
        self.buys = OrderTree(True)


    def __str__(self):
        return "---------------\nBuy Book \n---------------\n" + str(self.buys) + "\n" + "---------------\nSell Book \n---------------\n" + str(self.sells) 

    def submitOrder(self,order): #ensure that it checks that the price is still currently a valid price

        deleteNodes = []
        
        if order.isBuy: #submitting a buy order
            if order.price < self.sells.bestPrice(): #no order fill
                self.buys.insertOrder(order)
            else: #order fillable
                for existingOrders, currentNode in self.sells.head.in_order(): # fills order on root node

                    if order.price < currentNode.value:
                        break
                    updatedOrders = []

                    for existingOrder in existingOrders:
                        if existingOrder.unfilledVolume <= order.unfilledVolume:
                            #fill entire order
                            order.execute((existingOrder.unfilledVolume,existingOrder.price))
                            existingOrder.execute((existingOrder.unfilledVolume,existingOrder.price))
                        else:
                            #partial fill the order
                            existingOrder.execute((order.unfilledVolume,existingOrder.price)) #adds tuple of trade execution to the order and sets remaning volume to zero
                            order.execute((order.unfilledVolume,existingOrder.price))
                            updatedOrders.append(existingOrder)
                            break
                        if existingOrder.unfilledVolume > 0:
                            updatedOrders.append(existingOrder)

                    currentNode.orders = updatedOrders
                    if len(updatedOrders) == 0:
                        deleteNodes.append(currentNode)


                    if order.unfilledVolume <= 0:#order is completely filled
                        break
                if order.unfilledVolume > 0:#insert remaining unfilled volume onto the market
                    self.buys.insertOrder(order)

                for node in deleteNodes:
                    if node.parent == None: 
                        self.sells.head = self.sells.head.right
                        if self.sells.head != None:
                            self.sells.head.parent = None
                    else:
                        if node.right == None:
                            node.parent.left = node.right
                        else:
                            node.right.parent = node.parent
                            node.parent.left = node.right
                        



        else: #submitting a sell order
            if order.price > self.buys.bestPrice(): #no order fill
                self.sells.insertOrder(order)
            else: #order fillable
                for existingOrders, currentNode in self.buys.head.in_order(): # fills order on root node

                    if order.price > currentNode.value:
                        break
                    updatedOrders = []

                    for existingOrder in existingOrders:
                        if existingOrder.unfilledVolume <= order.unfilledVolume:
                            #fill entire order
                            order.execute((existingOrder.unfilledVolume,existingOrder.price))
                            existingOrder.execute((existingOrder.unfilledVolume,existingOrder.price))
                        else:
                            #partial fill the order
                            existingOrder.execute((order.unfilledVolume,existingOrder.price)) #adds tuple of trade execution to the order and sets remaning volume to zero
                            order.execute((order.unfilledVolume,existingOrder.price))
                            updatedOrders.append(existingOrder)
                            break
                        if existingOrder.unfilledVolume > 0:
                            updatedOrders.append(existingOrder)

                    currentNode.orders = updatedOrders
                    if len(updatedOrders) == 0:
                        deleteNodes.append(currentNode)


                    if order.unfilledVolume <= 0:#order is completely filled
                        break

                if order.unfilledVolume > 0: #insert remaining unfilled volume onto the market
                    self.sells.insertOrder(order)
        
                for node in deleteNodes:
                    if node.parent == None: 
                        self.buys.head = self.buys.head.right
                        if self.buys.head != None:
                            self.buys.head.parent = None
                    else:
                        if node.right == None:
                            node.parent.left = node.right
                        else:
                            node.right.parent = node.parent
                            node.parent.left = node.right\
                            
    def cancelOrder(self):
        pass

In [ ]:
class member:
    def __init__(self,cash):
        self.cash = cash
        self.position = 0
        self.orders = []

In [859]:
exchange = orderbook()

order1 = Order(100,40,True)
order2 = Order(120,40,True)


exchange.submitOrder(order1)
exchange.submitOrder(order2)

a = Order(150,40,False)
exchange.submitOrder(a)


a = Order(100,60,False)
exchange.submitOrder(a)

print(exchange)


---------------
Buy Book 
---------------
bestPrice=100
Owner=None, Price=100, Volume=40, Unfilled=20
---------------
Sell Book 
---------------
bestPrice=150
Owner=None, Price=150, Volume=40, Unfilled=40


In [860]:
import random
exchange = orderbook()

for x in range(100):
    
    order1 = Order(np.floor(np.random.normal()* 25) + 500,random.randint(1,500),False)
    order2 = Order(np.floor(np.random.normal()* 25) + 495,random.randint(1,500),True)
    exchange.submitOrder(random.choice([order1,order2]))


'''
for x in range(1000):
    try:
        mid_price = int((exchange.sells.bestPrice() + exchange.buys.bestPrice())/2)
    except:
        mid_price = 500

    
    order1 = Order(np.floor(np.random.normal()* 25) + mid_price+2,random.randint(1,500),False)
    order2 = Order(np.floor(np.random.normal()* 25) + mid_price-2,random.randint(1,500),True)
    exchange.submitOrder(random.choice([order1,order2]))
'''

print(exchange)

---------------
Buy Book 
---------------
bestPrice=488.0
Owner=None, Price=488.0, Volume=462, Unfilled=31.
Owner=None, Price=487.0, Volume=43, Unfilled=43.
Owner=None, Price=487.0, Volume=357, Unfilled=357.
Owner=None, Price=485.0, Volume=212, Unfilled=212.
Owner=None, Price=481.0, Volume=30, Unfilled=30.
Owner=None, Price=479.0, Volume=48, Unfilled=48.
Owner=None, Price=477.0, Volume=383, Unfilled=383.
Owner=None, Price=476.0, Volume=155, Unfilled=155.
Owner=None, Price=470.0, Volume=218, Unfilled=218.
Owner=None, Price=461.0, Volume=169, Unfilled=169.
Owner=None, Price=458.0, Volume=54, Unfilled=54.
Owner=None, Price=457.0, Volume=45, Unfilled=45.
Owner=None, Price=452.0, Volume=436, Unfilled=436.
Owner=None, Price=452.0, Volume=432, Unfilled=432.
Owner=None, Price=446.0, Volume=137, Unfilled=137
---------------
Sell Book 
---------------
bestPrice=489.0
Owner=None, Price=489.0, Volume=442, Unfilled=201.
Owner=None, Price=493.0, Volume=484, Unfilled=484.
Owner=None, Price=519.0, Vol

In [861]:
#generating a random exchange 

# Buy side (sorted high to low)


buy_prices = []
buy_volumes = []

for orders, currentNode in exchange.buys.head.in_order():
    buy_prices.append(currentNode.value)
    buy_volumes.append(sum(order.unfilledVolume for order in orders))

# Sell side (sorted low to high)


sell_prices = []
sell_volumes = []

for orders, currentNode in exchange.sells.head.in_order():
    sell_prices.append(currentNode.value)
    sell_volumes.append(sum(order.unfilledVolume for order in orders))


buy_cumvol = np.cumsum(buy_volumes) 
sell_cumvol = np.cumsum(sell_volumes)           

fig = go.Figure()

# Buy side (bids)
fig.add_trace(go.Scatter(
    x=buy_prices,
    y=buy_cumvol,
    mode='lines+markers',
    name='Bids (Buy)',
    line=dict(color='green', shape='hv')
))

# Sell side (asks)
fig.add_trace(go.Scatter(
    x=sell_prices,
    y=sell_cumvol,
    mode='lines+markers',
    name='Asks (Sell)',
    line=dict(color='red', shape='hv')
))

fig.update_layout(
    title='Order Book Depth Chart',
    xaxis_title='Price',
    yaxis_title='Cumulative Volume',
    xaxis=dict(type='linear'),
    yaxis=dict(type='linear'),
    template='plotly_white'
)

fig.show()


In [862]:
import plotly.graph_objects as go
import numpy as np


frames = []
initial_bids = []
initial_asks = []

for t in range(100):
    buy_prices = []
    buy_volumes = []
    for orders, currentNode in exchange.buys.head.in_order():
        buy_prices.append(currentNode.value)
        buy_volumes.append(sum(order.unfilledVolume for order in orders))
    
    sell_prices = []
    sell_volumes = []
    for orders, currentNode in exchange.sells.head.in_order():
        sell_prices.append(currentNode.value)
        sell_volumes.append(sum(order.unfilledVolume for order in orders))

    buy_cumvol = np.cumsum(buy_volumes)
    sell_cumvol = np.cumsum(sell_volumes)

    # Save the first snapshot to use as the base
    if t == 0:
        initial_bids = [go.Scatter(
            x=buy_prices,
            y=buy_cumvol,
            mode='lines+markers',
            name='Bids (Buy)',
            line=dict(color='green', shape='hv')
        )]
        initial_asks = [go.Scatter(
            x=sell_prices,
            y=sell_cumvol,
            mode='lines+markers',
            name='Asks (Sell)',
            line=dict(color='red', shape='hv')
        )]

    # Each frame stores updated bid/ask lines
    frames.append(go.Frame(
        name=str(t),
        data=[
            go.Scatter(x=buy_prices, y=buy_cumvol, mode='lines+markers',
                       line=dict(color='green', shape='hv')),
            go.Scatter(x=sell_prices, y=sell_cumvol, mode='lines+markers',
                       line=dict(color='red', shape='hv'))
        ]
    ))

    order1 = Order(np.floor(np.random.normal()* 10) + 500,random.randint(1,500),False)
    order2 = Order(np.floor(np.random.normal()* 10) + 495,random.randint(1,500),True)
    exchange.submitOrder(random.choice([order1,order2]))

# Create figure with initial data
fig = go.Figure(
    data=initial_bids + initial_asks,
    layout=go.Layout(
        title="Order Book Depth Chart Over Time",
        xaxis_title="Price",
        yaxis_title="Cumulative Volume",
        template='plotly_white',
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play", method="animate", args=[None])]
        )]
    ),
    frames=frames
)

# Add slider for time
fig.update_layout(
    sliders=[{
        "steps": [
            {
                "method": "animate",
                "args": [[str(t)], {"mode": "immediate", "frame": {"duration": 200, "redraw": True}}],
                "label": f"t={t}"
            } for t in range(len(frames))
        ],
        "transition": {"duration": 0},
        "x": 0.1,
        "len": 0.9
    }]
)

fig.show()
